In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


In [2]:

data=pd.read_csv('spam_clean.csv',encoding='latin-1')
data.head()

,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# cleaning messages

In [3]:
import nltk 
import re
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\SONY\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SONY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def clean_message(sentence):
  words=nltk.word_tokenize(sentence)
  cleaned_msg=""
  for word in words:
    word=word.lower()
    word=re.sub(r'[^a-z]','',word)
    if word!="" and word not in stopwords.words('english'):
      cleaned_msg=cleaned_msg+" "+word
  return cleaned_msg.strip()
  
    

In [5]:
data['cleaned_msg']=data['message'].apply(clean_message)
data.head()

In [ ]:
data.shape

(5572, 3)

# data preparation for naive bayes

In [ ]:
class NaiveBayes:
    def __init__(self):
        self.common_words=[]

    
    def fit(self,X,y,common_words=10):
        self.X=X
        self.y=y
        ham=""
        spam=""
        ham=ham.join(self.X[self.y=='ham'])
        ham_words=ham.split()
        spam=spam.join(self.X[self.y=='spam'])
        spam_words=spam.split()

        ham_word_count=np.array(Counter(ham_words).most_common(common_words))[:,0]
        spam_word_count=np.array(Counter(spam_words).most_common(common_words))[:,0]

        self.features=ham_word_count
        self.features=np.append(self.features,spam_word_count)
        self.features=np.unique(self.features)
        

        count_matrix=pd.DataFrame(np.zeros((self.X.shape[0],self.features.shape[0])))
        count_matrix.columns=self.features

        self.word_frequency_matrix= pd.concat([self.X,count_matrix],axis=1)
        print(self.word_frequency_matrix.shape)
        self.word_count()
        return self.word_frequency_matrix
    
    def word_count(self):
        for idx,row in enumerate(self.word_frequency_matrix['cleaned_msg']):
            for feature in self.features:
                self.word_frequency_matrix.loc[idx,feature]=row.count(feature)
                 

In [ ]:
NB=NaiveBayes()
NB.fit(data['cleaned_msg'],data['type'])

(5572, 18)


,cleaned_msg,call,claim,free,get,go,got,gt,like,lt,mobile,nt,reply,stop,text,txt,u,ur
0,go jurong point crazy available bugis n great ...,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0
1,ok lar joking wif u oni,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,free entry wkly comp win fa cup final tkts st ...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,2.0,0.0
3,u dun say early hor u c already say,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,nah nt think goes usf lives around though,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,nd time tried contact u u pound prize claim ea...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0
5568,b going esplanade fr home,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,pity mood suggestions,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5570,guy bitching acted like interested buying some...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0


In [ ]:
data.iloc[2,2]

'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply'